In [1]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

The only thing we know about the data is we are trying to use x variables to predict y variable.

In [2]:
my_df = pd.read_csv('../../../Data/multi-set-1.csv')

In [3]:
my_df.info

<bound method DataFrame.info of      Unnamed: 0        X0        X1        X2        X3        X4        X5  \
0             0 -2.322018 -0.347711 -0.613853 -0.283326 -1.825783 -0.832817   
1             1  0.694377 -0.244357 -1.135003  0.343134 -0.579199  0.232848   
2             2 -1.345888 -0.727013  1.645505 -0.675970  0.358269  2.949318   
3             3  1.042922 -0.623371 -0.809288 -0.406957 -0.084170 -0.669237   
4             4  0.588704 -0.834573 -1.597467  1.919520  0.172437  2.413028   
..          ...       ...       ...       ...       ...       ...       ...   
745         745 -1.471496  1.013023  0.503969  1.097987  1.541199  0.643162   
746         746 -0.031940  1.582798  0.284986 -0.686531 -1.544323 -0.927135   
747         747  0.556434 -1.081989  2.005101  0.081933  0.748353  0.657440   
748         748  0.822929  2.932668 -1.118101 -1.805829  0.353588 -0.658435   
749         749 -1.395976 -0.929219 -0.548496 -0.501613 -0.916014  0.569704   

           X6      

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
y = my_df['y']

In [6]:
X = my_df.drop('y', axis=1)

In [7]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

Split into training and test sets.

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.7, random_state=1)

In [9]:
feature_selector = SelectKBest(score_func=f_regression, k=3)

Feature selector below uses a criterion called "f_regression", and selects the three best features according to that criterion.

In [10]:
feature_selector

SelectKBest(k=3, score_func=<function f_regression at 0x1BABA460>)

In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

Pipeline is first going to use feature_selector to select the 3 best features according to f_regression and then fit a LInear Regression model to it.

In [12]:
lm = LinearRegression()

In [13]:
pl = make_pipeline(feature_selector, lm)

In [14]:
pl.fit(X_train, y_train)

Pipeline(steps=[('selectkbest',
                 SelectKBest(k=3,
                             score_func=<function f_regression at 0x1BABA460>)),
                ('linearregression', LinearRegression())])

In [15]:
lm.coef_

array([ 1.99766284, -2.09379528, -1.5218715 ])

In [16]:
lm.intercept_

0.007164358096600668

In [17]:
pl.score(X_train, y_train)

0.8562958252685078

We should take this and see how it scores on the test data.

In [18]:
pl.score(X_test, y_test)

0.8186188695223429

Model does worse on testing then training thus slight evidence there might be over fitting.

We would like to automate this.

In [19]:
def make_k_best_features_f_regression_pipeline(X, y, num_features):
    # split data into train/test
    X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                        train_size=0.7,
                                                        random_state=1)
    # created SelectKBestFeatures object that used f-regression
    feature_selector = SelectKBest(score_func=f_regression, k=num_features)

    # Created a linearRegression object
    lm = LinearRegression()
    pl = make_pipeline(feature_selector,
                       lm)

    # fit data
    pl.fit(X_train, y_train)

    # Created a pipeline
    return pl

In [20]:
make_k_best_features_f_regression_pipeline(X, y, 4)

Pipeline(steps=[('selectkbest',
                 SelectKBest(k=4,
                             score_func=<function f_regression at 0x1BABA460>)),
                ('linearregression', LinearRegression())])

Now we are going to create a for loop to compare pipelines.

In [27]:
for j in range(1, 9):
    my_pl_w_j_features = make_k_best_features_f_regression_pipeline(X, y, j)

    train_score_j = my_pl_w_j_features.score(X_train, y_train)
    test_score_j = my_pl_w_j_features.score(X_test, y_test)

    print("With " + str(j) + " features training set score is: ",
          train_score_j, "\n            and the test set score is: ", test_score_j)
    print("\n")

With 1 features training set score is:  0.3686364807385255 
            and the test set score is:  0.2602569314848532


With 2 features training set score is:  0.6964040974121528 
            and the test set score is:  0.6228953327159134


With 3 features training set score is:  0.8562958252685078 
            and the test set score is:  0.8186188695223429


With 4 features training set score is:  0.926596512257632 
            and the test set score is:  0.9188802144103643


With 5 features training set score is:  0.9266398423325634 
            and the test set score is:  0.9188404788951915


With 6 features training set score is:  0.9266460620392583 
            and the test set score is:  0.9187788457356857


With 7 features training set score is:  0.9266461254408862 
            and the test set score is:  0.9187548757770102


With 8 features training set score is:  0.9266593717277892 
            and the test set score is:  0.9188765912377325


